# Scenario Generator

This jupyter based interface has been designed to support scenario runs for the Ethiopia PATHWAYS project. 

The interface is built upon [OnSSET](http://www.onsset.org/) developed by KTH division of Energy Systems to provide an easy and quick way to generate electrification investment scenarios for the Ethiopia PATHWAYS project. 

#### Start by importing the code 

In [1]:
from onsset import *
from IPython.display import display, Markdown, HTML
%matplotlib inline

2020-01-23 12:10:28,246		CACHEDIR=C:\Users\asahl\.matplotlib
2020-01-23 12:10:28,252		Using fontManager instance from C:\Users\asahl\.matplotlib\fontlist-v300.json
2020-01-23 12:10:28,378		Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-01-23 12:10:28,381		Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-01-23 12:10:28,387		Loaded backend module://ipykernel.pylab.backend_inline version unknown.


# 1. GIS data selection

First, run the cell below to browse to the directory where your calibrated input CSV file is located at and select the input file.


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)
messagebox.showinfo('OnSSET', 'Open the input file with extracted GIS data')
input_file = filedialog.askopenfilename()

onsseter = SettlementProcessor(input_file)
onsseter.df['IsUrban'] = 0
onsseter.df['Conflict'] = 0
onsseter.df['PerCapitaDemand'] = 0

# 2. Modelling period and target electrification rate

Next, define the modelling period and the electrification rate to be achieved by the years of the analysis.

In [3]:
start_year = 2018
end_year = 2070
yearsofanalysis = [2025, 2030, 2040, 2050, 2060, 2070]
time_steps = {2025: 7, 
              2030: 5, 
              2040: 10, 
              2050: 10, 
              2060: 10, 
              2070: 10}

# Electrification rate target. E.g. 1 = 100%, 0.5 = 50%
eleclimits = {2025: 1, 
              2030: 1, 
              2040: 1, 
              2050: 1, 
              2060: 1, 
              2070: 1}

In [4]:
# This is the maximum generation capacity (in MW) that can be added to the grid in one year. 
# "999999999" means that it is effectively unlimited.
annual_grid_cap_gen_limit = {2025: 999999999, 
                             2030: 999999999, 
                             2040: 999999999, 
                             2050: 999999999, 
                             2060: 999999999, 
                             2070: 999999999}       

In [5]:
# Buffer distance (km) from the current grid network for automatic connection to the grid. 
# 5 km reflects approximately 65% of the population, 25 km reflects approximately 95% of the population. 
# 0 means that there is no automatic connection to the grid, each settlement will consider both on- and off-grid alternatives
auto_intensification = {2025: 0, 
                        2030: 0, 
                        2040: 0, 
                        2050: 0, 
                        2060: 0, 
                        2070: 0}

In [6]:
# This is the maximum amount of new households that can be connected to the grid in one year (thousands).
# "999999999" means that it is effectively unlimited.
annual_new_grid_connections_limit = {2025: 300, 
                                     2030: 999999999, 
                                     2040: 999999999, 
                                     2050: 999999999, 
                                     2060: 999999999, 
                                     2070: 999999999} 

In [7]:
num_people_per_hh_rural = 5.2
num_people_per_hh_urban = 4.4

### b. Technology specifications & costs

The cell below contains all the information that is used to calculate the levelized costs for all the technologies, including grid. These default values should be updated to reflect the most accurate values in the country. There are currently 7 potential technologies to include in the model:
* Grid
* PV Mini-grid
* Wind Mini-grid
* Hydro Mini-grid
* Diesel Mini-grid
* PV Stand-alone systems
* Diesel Stand-alone systems

First, decide whether to include diesel technologies or not:

In [8]:
diesel_techs = 0                      ### 0 = diesel NOT included, 1 = diesel included 

The grid generation cost and grid power plants capital cost below for each time-step is given from the OSeMOSYS analysis

In [16]:
### This is the grid cost electricity USD/kWh as expected in the years of the analysis
grid_generation_cost = {2025: 0.11, 
                        2030: 0.11, 
                        2040: 0.11, 
                        2050: 0.11, 
                        2060: 0.11, 
                        2070: 0.11}

### The cost in USD/kW to for power generation capacity additions to the grid
grid_power_plants_capital_cost = {2025: 2248, 
                                  2030: 2248, 
                                  2040: 2248, 
                                  2050: 2248, 
                                  2060: 2248, 
                                  2070: 2248}

In [17]:
### The fraction of electricity lost in transmission and distribution (percentage) 
grid_losses = {2025: 0.122, 
               2030: 0.122, 
               2040: 0.122, 
               2050: 0.122, 
               2060: 0.122, 
               2070: 0.122}                     

In [18]:
### This is the diesel price in USD/liter as expected in the years of the analysis.
diesel_price = {2025: 0.85, 
                2030: 0.85, 
                2040: 0.85, 
                2050: 0.85, 
                2060: 0.85, 
                2070: 0.85}                   

In [19]:
### Stand-alone Diesel capital cost (USD/kW) as expected in the years of the analysis
sa_diesel_capital_cost = {2025: 938, 
                          2030: 938, 
                          2040: 938, 
                          2050: 938, 
                          2060: 938, 
                          2070: 938}

### Mini-grid Diesel capital cost (USD/kW) as expected in the years of the analysis
mg_diesel_capital_cost = {2025: 721, 
                          2030: 721, 
                          2040: 721, 
                          2050: 721, 
                          2060: 721, 
                          2070: 721}

### Mini-grid PV capital cost (USD/kW) as expected in the years of the analysis
mg_pv_capital_cost = {2025: 2950, 
                      2030: 2950, 
                      2040: 2950, 
                      2050: 2950, 
                      2060: 2950, 
                      2070: 2950}

### Mini-grid Wind capital cost (USD/kW) as expected in the years of the analysis
mg_wind_capital_cost = {2025: 3750, 
                        2030: 3750, 
                        2040: 3750, 
                        2050: 3750, 
                        2060: 3750, 
                        2070: 3750}

### Mini-grid Hydro capital cost (USD/kW) as expected in the years of the analysis
mg_hydro_capital_cost = {2025: 3000, 
                         2030: 3000, 
                         2040: 3000, 
                         2050: 3000, 
                         2060: 3000, 
                         2070: 3000}          

In [20]:
### Stand-alone PV capital cost (USD/kW) for household systems under 20 W
sa_pv_capital_cost_1 = {2025: 9620, 
                        2030: 9620, 
                        2040: 9620, 
                        2050: 9620, 
                        2060: 9620, 
                        2070: 9620}
### Stand-alone PV capital cost (USD/kW) for household systems between 21-50 W
sa_pv_capital_cost_2 = {2025: 8780, 
                        2030: 8780, 
                        2040: 8780, 
                        2050: 8780, 
                        2060: 8780, 
                        2070: 8780}
### Stand-alone PV capital cost (USD/kW) for household systems between 51-100 W
sa_pv_capital_cost_3 = {2025: 6380, 
                        2030: 6380, 
                        2040: 6380, 
                        2050: 6380, 
                        2060: 6380, 
                        2070: 6380}
### Stand-alone PV capital cost (USD/kW) for household systems between 101-1000 W
sa_pv_capital_cost_4 = {2025: 4470, 
                        2030: 4470, 
                        2040: 4470, 
                        2050: 4470, 
                        2060: 4470, 
                        2070: 4470}
### Stand-alone PV capital cost (USD/kW) for household systems over 1 kW
sa_pv_capital_cost_5 = {2025: 6950, 
                        2030: 6950, 
                        2040: 6950, 
                        2050: 6950, 
                        2060: 6950, 
                        2070: 6950}           

The cells below contain additional technology specifications

In [21]:
discount_rate = 0.10                  ### E.g. 0.10 means a discount rate of 10%

# Transmission and distribution costs
existing_grid_cost_ratio = 0.1        ### The additional cost per round of electrification (percentage)
hv_line_capacity=69 # kV
hv_line_cost=53000 # USD/km
mv_line_cost = 7000 # USD/kW
mv_line_capacity=50 # kV
mv_line_max_length=50 # km
mv_increase_rate=0.1
max_mv_line_dist = 50 # km
MV_line_amperage_limit = 8  # Ampere (A)
lv_line_capacity=0.24 #kV
lv_line_max_length=0.8 # km
lv_line_cost=4250 # USD/km
service_Transf_type=50  # kVa
service_Transf_cost=4250  # $/unit
max_nodes_per_serv_trans=300  # maximum number of nodes served by each service transformer
hv_lv_transformer_cost=25000 # USD/unit
hv_mv_transformer_cost=25000 # USD/unit
mv_lv_transformer_cost=10000 # USD/unit
mv_mv_transformer_cost=10000 # USD/unit

Technology.set_default_values(base_year=start_year, start_year=start_year, end_year=end_year, discount_rate=discount_rate,
                             HV_line_type=hv_line_capacity, HV_line_cost=hv_line_cost, MV_line_type=mv_line_capacity,
                             MV_line_amperage_limit=MV_line_amperage_limit, MV_line_cost=mv_line_cost, LV_line_type=lv_line_capacity,
                             LV_line_cost=lv_line_cost, LV_line_max_length=lv_line_max_length, 
                             service_Transf_type=service_Transf_type, service_Transf_cost = service_Transf_cost,
                             max_nodes_per_serv_trans=max_nodes_per_serv_trans, MV_LV_sub_station_cost=mv_lv_transformer_cost,
                             MV_MV_sub_station_cost=mv_mv_transformer_cost, HV_LV_sub_station_cost=hv_lv_transformer_cost,
                             HV_MV_sub_station_cost=hv_mv_transformer_cost)


# 7. Start a scenario run, which calculate and compare technology costs for every settlement in the country

Based on the previous calculation this piece of code identifies the LCoE that every off-grid technology can provide, for each single populated settlement of the selected country. The cell then takes all the currently grid-connected points in the country, and looks at the points within a certain distance from them, to see if it is more economical to connect them to the grid, or to use one of the off-grid technologies calculated above. Once more points are connected to the grid, the process is repeated, so that new points close to those points might also be connected. This is repeated until there are no new points to connect to the grid.

In [22]:
onsseter.current_mv_line_dist()

for year in yearsofanalysis:
    end_year_pop = 1
    prioritization = 2
        
    eleclimit = eleclimits[year]
    time_step = time_steps[year]
    
    # Centralized grid costs
    grid_calc = Technology(om_of_td_lines=0.02,
                        distribution_losses=grid_losses[year],
                        connection_cost_per_hh=125,
                        base_to_peak_load_ratio=0.8,
                        capacity_factor=1,
                        tech_life=30,
                        grid_capacity_investment=grid_power_plants_capital_cost[year],
                        grid_price=grid_generation_cost[year])

    # Mini-grid hydro costs
    mg_hydro_calc = Technology(om_of_td_lines=0.02,
                            distribution_losses=0.05,
                            connection_cost_per_hh=100,
                            base_to_peak_load_ratio=0.85,
                            capacity_factor=0.5,
                            tech_life=30,
                            capital_cost=mg_hydro_capital_cost[year],
                            om_costs=0.03,
                            )

    # Mini-grid wind costs
    mg_wind_calc = Technology(om_of_td_lines=0.02,
                            distribution_losses=0.05,
                            connection_cost_per_hh=100,
                            base_to_peak_load_ratio=0.85,
                            capital_cost=mg_wind_capital_cost[year],
                            om_costs=0.02,
                            tech_life=20,
                            )

    # Mini-grid PV costs
    mg_pv_calc = Technology(om_of_td_lines=0.02,
                        distribution_losses=0.05,
                        connection_cost_per_hh=100,
                        base_to_peak_load_ratio=0.85,
                        tech_life=20,
                        om_costs=0.015,
                        capital_cost=mg_pv_capital_cost[year]                        
                       )

    # Stand-alone PV costs
    sa_pv_calc = Technology(base_to_peak_load_ratio=0.9,
                        tech_life=15,
                        om_costs=0.02,
                        capital_cost={0.020: sa_pv_capital_cost_1[year], 
                                      0.050: sa_pv_capital_cost_2[year], 
                                      0.100: sa_pv_capital_cost_3[year], 
                                      1: sa_pv_capital_cost_4[year], 
                                      5: sa_pv_capital_cost_5[year]},
                        standalone=True
                        )

    # Mini-grid diesel costs
    mg_diesel_calc = Technology(om_of_td_lines=0.02,
                            distribution_losses=0.05,
                            connection_cost_per_hh=100,
                            base_to_peak_load_ratio=0.85,
                            capacity_factor=0.7,
                            tech_life=15,
                            om_costs=0.1,
                            capital_cost=mg_diesel_capital_cost[year]
                            )

    # Stand-alone diesel costs
    sa_diesel_calc = Technology(base_to_peak_load_ratio=0.9,
                            capacity_factor=0.5,
                            tech_life=10,
                            om_costs=0.1,
                            capital_cost=sa_diesel_capital_cost[year],
                            standalone=True)
    
    sa_diesel_cost = {'diesel_price': diesel_price[year],
                          'efficiency': 0.28,
                          'diesel_truck_consumption': 14,
                          'diesel_truck_volume': 300}


    mg_diesel_cost = {'diesel_price':diesel_price[year],
                          'efficiency': 0.33,
                          'diesel_truck_consumption': 33.7,
                          'diesel_truck_volume': 15000}
    
    start_year = year - time_step
    end_year = year
    
    grid_cap_gen_limit = time_step * annual_grid_cap_gen_limit[year] * 1000
    grid_connect_limit = time_step * annual_new_grid_connections_limit[year] * 1000
    
    onsseter.set_scenario_variables(year, num_people_per_hh_rural, num_people_per_hh_urban, time_step,
                                            start_year, 6, 6, 1, 0)
    
    onsseter.diesel_cost_columns(sa_diesel_cost, mg_diesel_cost, year)

    onsseter.calculate_off_grid_lcoes(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                        sa_diesel_calc, year, start_year, end_year, time_step,
                                     diesel_techs=diesel_techs)

    onsseter.pre_electrification(grid_generation_cost[year], year, time_step, start_year)

    onsseter.df[SET_LCOE_GRID + "{}".format(year)], onsseter.df[SET_MIN_GRID_DIST + "{}".format(year)], onsseter.df[
                SET_ELEC_ORDER + "{}".format(year)], onsseter.df[SET_MV_CONNECT_DIST] = onsseter.elec_extension(grid_calc, 
                    max_mv_line_dist, year, start_year, end_year, time_step,
                    grid_cap_gen_limit, grid_connect_limit, auto_intensification=auto_intensification[year],
                    prioritization=prioritization)

    onsseter.results_columns(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                     sa_diesel_calc, grid_calc, year)

    onsseter.calculate_investments(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                           sa_diesel_calc, grid_calc, year, end_year, time_step)

    onsseter.apply_limitations(eleclimit, year, time_step, prioritization, auto_intensification[year])

    onsseter.final_decision(year)
    
    onsseter.calculate_new_capacity(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                            sa_diesel_calc, grid_calc, year, time_step)

    onsseter.pv_system_type(year, sa_pv_calc)

2020-01-23 13:37:49,941		Determine current MV line length
2020-01-23 13:37:50,104		Calculate new connections
2020-01-23 13:37:50,119		Setting electrification demand as per target per year
2020-01-23 13:37:52,122		Calculate minigrid hydro LCOE
2020-01-23 13:37:53,785		273 potential hydropower sites were utilised to maximum capacity
2020-01-23 13:37:53,785		Calculate minigrid PV LCOE
2020-01-23 13:37:58,364		Calculate minigrid wind LCOE
2020-01-23 13:38:03,188		Calculate minigrid diesel LCOE
2020-01-23 13:38:08,113		Calculate standalone PV LCOE
2020-01-23 13:38:13,208		Determine minimum technology (off-grid)
2020-01-23 13:38:13,241		Determine minimum tech LCOE
2020-01-23 13:38:13,278		Define the initial electrification status
2020-01-23 13:38:17,187		Initially 1352 electrified
2020-01-23 13:38:18,213		Electrification loop 1 with 4217 electrified
2020-01-23 13:38:18,540		Electrification loop 2 with 460 electrified
2020-01-23 13:38:18,708		Electrification loop 3 with 28 electrified
2020-01

The electrification rate achieved in 2025 is 0.0 %


2020-01-23 13:38:25,209		Calculate HV investment cost
2020-01-23 13:38:28,332		Calculate new connections
2020-01-23 13:38:28,343		Setting electrification demand as per target per year
2020-01-23 13:38:30,472		Calculate minigrid hydro LCOE
2020-01-23 13:38:31,995		319 potential hydropower sites were utilised to maximum capacity
2020-01-23 13:38:31,995		Calculate minigrid PV LCOE
2020-01-23 13:38:36,140		Calculate minigrid wind LCOE
2020-01-23 13:38:40,712		Calculate minigrid diesel LCOE
2020-01-23 13:38:45,064		Calculate standalone PV LCOE
2020-01-23 13:38:49,326		Determine minimum technology (off-grid)
2020-01-23 13:38:49,374		Determine minimum tech LCOE
2020-01-23 13:38:49,408		Define the initial electrification status
2020-01-23 13:38:53,132		Initially 6060 electrified
2020-01-23 13:38:54,047		Electrification loop 1 with 1842 electrified
2020-01-23 13:38:54,431		Electrification loop 2 with 303 electrified
2020-01-23 13:38:54,687		Electrification loop 3 with 57 electrified
2020-01-23 

The electrification rate achieved in 2030 is 0.0 %


2020-01-23 13:39:04,570		Calculate HV investment cost
2020-01-23 13:39:09,797		Calculate new connections
2020-01-23 13:39:09,808		Setting electrification demand as per target per year
2020-01-23 13:39:11,909		Calculate minigrid hydro LCOE
2020-01-23 13:39:13,664		432 potential hydropower sites were utilised to maximum capacity
2020-01-23 13:39:13,665		Calculate minigrid PV LCOE
2020-01-23 13:39:18,809		Calculate minigrid wind LCOE
2020-01-23 13:39:23,402		Calculate minigrid diesel LCOE
2020-01-23 13:39:28,106		Calculate standalone PV LCOE
2020-01-23 13:39:33,047		Determine minimum technology (off-grid)
2020-01-23 13:39:33,083		Determine minimum tech LCOE
2020-01-23 13:39:33,119		Define the initial electrification status
2020-01-23 13:39:36,803		Initially 8273 electrified
2020-01-23 13:39:38,923		Electrification loop 1 with 4792 electrified
2020-01-23 13:39:40,077		Electrification loop 2 with 1143 electrified
2020-01-23 13:39:40,745		Electrification loop 3 with 174 electrified
2020-01-2

...

2020-01-23 13:40:32,240		Calculate HV investment cost
2020-01-23 13:40:37,154		Calculate new connections
2020-01-23 13:40:37,165		Setting electrification demand as per target per year
2020-01-23 13:40:39,288		Calculate minigrid hydro LCOE
2020-01-23 13:40:41,028		497 potential hydropower sites were utilised to maximum capacity
2020-01-23 13:40:41,029		Calculate minigrid PV LCOE
2020-01-23 13:40:44,278		Calculate minigrid wind LCOE
2020-01-23 13:40:47,950		Calculate minigrid diesel LCOE
2020-01-23 13:40:51,359		Calculate standalone PV LCOE
2020-01-23 13:40:54,218		Determine minimum technology (off-grid)
2020-01-23 13:40:54,262		Determine minimum tech LCOE
2020-01-23 13:40:54,307		Define the initial electrification status
2020-01-23 13:40:56,454		Initially 22455 electrified
2020-01-23 13:40:58,680		Electrification loop 1 with 3558 electrified
2020-01-23 13:41:00,446		Electrification loop 2 with 1734 electrified
2020-01-23 13:41:01,333		Electrification loop 3 with 476 electrified
2020-01-

The electrification rate achieved in 2060 is 0.0 %


2020-01-23 13:41:13,033		Calculate HV investment cost
2020-01-23 13:41:18,907		Calculate new connections
2020-01-23 13:41:18,920		Setting electrification demand as per target per year
2020-01-23 13:41:21,057		Calculate minigrid hydro LCOE
2020-01-23 13:41:22,790		506 potential hydropower sites were utilised to maximum capacity
2020-01-23 13:41:22,791		Calculate minigrid PV LCOE
2020-01-23 13:41:25,213		Calculate minigrid wind LCOE
2020-01-23 13:41:27,785		Calculate minigrid diesel LCOE
2020-01-23 13:41:30,462		Calculate standalone PV LCOE
2020-01-23 13:41:32,348		Determine minimum technology (off-grid)
2020-01-23 13:41:32,388		Determine minimum tech LCOE
2020-01-23 13:41:32,428		Define the initial electrification status
2020-01-23 13:41:34,200		Initially 28452 electrified
2020-01-23 13:41:35,702		Electrification loop 1 with 1273 electrified
2020-01-23 13:41:36,688		Electrification loop 2 with 619 electrified
2020-01-23 13:41:37,470		Electrification loop 3 with 219 electrified
2020-01-2

The electrification rate achieved in 2070 is 0.0 %


2020-01-23 13:41:49,252		Calculate HV investment cost


# 8. Results, Summaries and Visualization
With all the calculations and grid-extensions complete, this block gets the final results on which technology was chosen for each point, how much capacity needs to be installed and what it will cost. Then the summaries, plots and maps are generated.

In [15]:
elements = []
for year in yearsofanalysis:
    elements.append("Population{}".format(year))
    elements.append("NewConnections{}".format(year))
    elements.append("Capacity{}".format(year))
    elements.append("Investment{}".format(year))

techs = ["Grid", "SA_Diesel", "SA_PV", "MG_Diesel", "MG_PV", "MG_Wind", "MG_Hydro"]

sumtechs = []
for year in yearsofanalysis:
    sumtechs.extend(["Population{}".format(year) + t for t in techs])
    sumtechs.extend(["NewConnections{}".format(year) + t for t in techs])
    sumtechs.extend(["Capacity{}".format(year) + t for t in techs])
    sumtechs.extend(["Investment{}".format(year) + t for t in techs])

summary = pd.Series(index=sumtechs, name='country')

for year in yearsofanalysis:
    for t in techs:
        summary.loc["Population{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)), SET_POP + '{}'.format(year)].sum()
        summary.loc["NewConnections{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)) & (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_NEW_CONNECTIONS + '{}'.format(year)].sum()
        summary.loc["Capacity{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)) & (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_NEW_CAPACITY + '{}'.format(year)].sum()/1000
        summary.loc["Investment{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)) & (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_INVESTMENT_COST + '{}'.format(year)].sum()
        
index = techs + ['Total']
columns = []
for year in yearsofanalysis:
    columns.append("Population{}".format(year))
    columns.append("NewConnections{}".format(year))
    columns.append("Capacity{} (MW)".format(year))
    columns.append("Investment{} (million USD)".format(year))
                                                                                                                                           
summary_table = pd.DataFrame(index=index, columns=columns)
turn = 0
for years in yearsofanalysis:
    summary_table[columns[turn * 4]] = summary.iloc[turn * 28:turn * 28 + 7].astype(int).tolist() + [int(summary.iloc[turn * 28:turn * 28 + 7].sum())]
    summary_table[columns[turn * 4 + 1]] = summary.iloc[turn * 28 + 7:turn * 28 + 14].astype(int).tolist() + [int(summary.iloc[turn * 28 + 7:turn * 28 + 14].sum())]
    summary_table[columns[turn * 4 + 2]] = summary.iloc[turn * 28 + 14:turn * 28 + 21].astype(int).tolist() + [int(summary.iloc[turn * 28 + 14:turn * 28 + 21].sum())]
    summary_table[columns[turn * 4 + 3]] = [round(x/1e4)/1e2 for x in summary.iloc[turn * 28 + 21:turn * 28 + 28].astype(float).tolist()] + [round(summary.iloc[turn * 28 + 21:turn * 28 + 28].sum()/1e4)/1e2]
    turn += 1


In [16]:
display(Markdown('### Summary \n These are the summarized results for full electrification of the selected country by the final year'))
summary_table

### Summary 
 These are the summarized results for full electrification of the selected country by the final year

,Population2025,NewConnections2025,Capacity2025 (MW),Investment2025 (million USD),Population2030,NewConnections2030,Capacity2030 (MW),Investment2030 (million USD),Population2040,NewConnections2040,...,Capacity2050 (MW),Investment2050 (million USD),Population2060,NewConnections2060,Capacity2060 (MW),Investment2060 (million USD),Population2070,NewConnections2070,Capacity2070 (MW),Investment2070 (million USD)
Grid,79615102,39827784,1896,6902.53,101137533,61141260,3319,10645.71,144989472,104161936,...,11385,33210.21,209547031,168639468,15812,44652.96,236426484,195517918,20133,55711.52
SA_Diesel,0,0,0,0.00,0,0,0,0.00,0,0,...,0,0.00,0,0,0,0.00,0,0,0,0.00
SA_PV,42569864,41617197,89,700.09,33930258,33171954,59,448.84,9315296,9260831,...,79,356.12,86582,86347,10,47.67,318,224,0,0.45
MG_Diesel,0,0,0,0.00,0,0,0,0.00,0,0,...,0,0.07,249,249,0,0.05,131,131,0,0.04
MG_PV,1544515,1536499,66,260.34,3055616,3052599,163,607.63,8526250,8517466,...,979,3144.70,6673387,6672741,1534,4852.89,6952109,6952063,2371,7349.77
MG_Wind,0,0,0,0.00,0,0,0,0.00,69,69,...,0,0.00,0,0,0,0.00,54,54,0,0.06
MG_Hydro,1097687,792632,30,132.50,1496771,1201309,55,216.18,3728805,3566533,...,205,804.53,4132069,3987457,398,1395.30,3999936,3855586,627,2088.49
Total,124827169,83774113,2082,7995.46,139620179,98567123,3598,11918.36,166559893,125506837,...,12650,37515.64,220439321,179386264,17757,50948.87,247379034,206325978,23133,65150.34


In [17]:
df_in = onsseter.df
df_in['PVType2018'] = 1
df_in['NewConnections2018'] = 0
df_in['NewCapacity2018'] = 0
df_in['InvestmentCost2018'] = 0
df_in['TransmissionInvestmentCost2018'] = 0
df_in['TotalEnergyPerCell2018'] = df_in['ElecPopCalib'] * df_in['ResidentialDemandTierCustom2018'] * df_in[SET_ELEC_CURRENT]
SET_TRANSMISSION_INV = 'TransmissionInvestmentCost'

elements = ["1.Population", "2.New_Connections", "3.Capacity", "4.Investment", "5. Demand", "6. Transmission summaries", "7. Distribution summaries", "8. Capacity factor"]
techs = ["Grid", "MG_Diesel", "MG_PV", "MG_Wind", "MG_Hydro", "SA_PV_1", "SA_PV_2", "SA_PV_3", "SA_PV_4", "SA_PV_5"]
years = [2018, 2025, 2030, 2040, 2050, 2060, 2070]
tech_codes = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12]
tech_costs = {1: 2248, 4: 721, 5: 2950, 6: 3750, 7: 3000, 8: 9620, 9: 8780, 10: 6380, 11: 4470, 12: 6950}


for year in years:
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 1), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 8
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 2), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 9
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 3), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 10
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 4), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 11
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 5), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 12

df_electrified = df_in.loc[df_in[SET_ELEC_CURRENT] == 1]
df_unelectrified = df_in.loc[df_in[SET_ELEC_CURRENT] == 0]

sumtechs = []
for element in elements:
    for tech in techs:
        sumtechs.append(element + "_" + tech)

total_rows = len(sumtechs)

electrified_sumtechs = pd.DataFrame(columns=years)
unelectrified_sumtechs = pd.DataFrame(columns=years)
combined_sumtechs = pd.DataFrame(columns=years)

for row in range(0, total_rows):
    electrified_sumtechs.loc[sumtechs[row]] = 0
    unelectrified_sumtechs.loc[sumtechs[row]] = 0
    combined_sumtechs.loc[sumtechs[row]] = 0

i = 0

# Adding pop summary (Million ppl
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_POP + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_POP + "{}".format(year)])/1000000
        combined_sumtechs[year][sumtechs[i]] = sum(df_in.loc[df_in[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_POP + "{}".format(year)]) / 1000000
    i += 1

# Adding connections summary (Million ppl)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CONNECTIONS + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CONNECTIONS + "{}".format(year)])/1000000
    i += 1

# Adding capacity summaries (MW)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CAPACITY + "{}".format(year)])/1000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CAPACITY + "{}".format(year)])/1000
    i += 1

# Adding investment summaries (Million USD)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_INVESTMENT_COST + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_INVESTMENT_COST + "{}".format(year)])/1000000
    i += 1

# Adding demand summaries (GWh)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_TOTAL_ENERGY_PER_CELL + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_TOTAL_ENERGY_PER_CELL + "{}".format(year)])/1000000
    i += 1

# Adding transmission summaries (Million USD)
for code in tech_codes:
    for year in years:
        if code == 1:
            electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code, SET_TRANSMISSION_INV + "{}".format(year)]) / 1000000
            unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code, SET_TRANSMISSION_INV + "{}".format(year)])/1000000
    i += 1

# Adding distribution summaries (USD/kW)
for code in tech_codes:
    for year in years:
        if electrified_sumtechs[year][sumtechs[i-40]] > 0:
            electrified_sumtechs[year][sumtechs[i]] = (electrified_sumtechs[year][sumtechs[i-30]]*1000000 -
                                                       electrified_sumtechs[year][sumtechs[i-10]]*1000000 -
                                                       electrified_sumtechs[year][sumtechs[i-40]]*1000 * tech_costs[code])/(electrified_sumtechs[year][sumtechs[i-40]]*1000)
        if unelectrified_sumtechs[year][sumtechs[i-40]] > 0:
            unelectrified_sumtechs[year][sumtechs[i]] = (unelectrified_sumtechs[year][sumtechs[i-30]]*1000000 -
                                                       unelectrified_sumtechs[year][sumtechs[i-10]]*1000000 -
                                                       unelectrified_sumtechs[year][sumtechs[i-40]]*1000 * tech_costs[code])/(unelectrified_sumtechs[year][sumtechs[i-40]]*1000)
    i += 1

for code in tech_codes:
    for year in years:
        if code == 5 or code > 7:
            if electrified_sumtechs[year][sumtechs[i-40]] > 0:
                electrified_sumtechs[year][sumtechs[i]] = df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_GHI].mean()/8760
            if unelectrified_sumtechs[year][sumtechs[i - 40]] > 0:
                unelectrified_sumtechs[year][sumtechs[i]] = df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_GHI].mean() / 8760
        elif code == 6:
            if electrified_sumtechs[year][sumtechs[i - 40]] > 0:
                electrified_sumtechs[year][sumtechs[i]] = df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_WINDCF].mean()
            if unelectrified_sumtechs[year][sumtechs[i - 40]] > 0:
                unelectrified_sumtechs[year][sumtechs[i]] = df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_WINDCF].mean()
    i += 1

all_sumtechs = electrified_sumtechs + unelectrified_sumtechs

## 9. Exporting results

This code generates five csv files:
 - one containing all the results for the scenario created
 - three containing the summaries for the scenario created
 - one containing some if the key input variables of the scenario

Before we proceed, please write the scenario_name in the first cell below. then move on to the next cell and run it to browse to the directory where you want to save your results. Sample file shall be located at .\ gep-onsset\sample_output. 

**Note that if you do not change the scenario name, the previous output files will be overwritten**

In [18]:
scenario_name = "scenario_3"

In [19]:
list1 = [('Start_year',start_year,'','',''), 
         ('End_year',yearsofanalysis,'','',''),
         ('End year electrification rate target',eleclimits,'','',''),
         ('Prioritization', prioritization,'','',''),
         ('Auto intensification distance', auto_intensification, '', '', 'Buffer distance (km) for automatic intensification if choosing prioritization 1'),
         ('discount_rate',discount_rate,'','',''),
         ('num_people_per_hh_urban',num_people_per_hh_urban,'','','the number of people per household expected in the end year (e.g. 2030)'),
         ('num_people_per_hh_rural',num_people_per_hh_rural,'','','the number of people per household expected in the end year (e.g. 2030)'),
         ('grid_generation_cost',grid_generation_cost,'','','This is the grid cost electricity USD/kWh as expected in the end year of the analysis'),
         ('grid_power_plants_capital_cost',grid_power_plants_capital_cost,'','','The cost in USD/kW to for capacity upgrades of the grid-connected power plants'),
         ('grid_losses',grid_losses,'','','The fraction of electricity lost in transmission and distribution (percentage)'),
         ('existing_grid_cost_ratio',existing_grid_cost_ratio,'','','The additional cost per round of electrification (percentage)'),
         ('diesel_price',diesel_price,'','','This is the diesel price in USD/liter as expected in the end year of the analysis'),
         ('sa_diesel_capital_cost',sa_diesel_capital_cost,'','','Stand-alone Diesel capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_diesel_capital_cost',mg_diesel_capital_cost,'','','Mini-grid Diesel capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_pv_capital_cost',mg_pv_capital_cost,'','','Mini-grid PV capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_wind_capital_cost',mg_wind_capital_cost,'','','Mini-grid Wind capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_hydro_capital_cost',mg_hydro_capital_cost,'','','Mini-grid Hydro capital cost (USD/kW) as expected in the years of the analysis'),
         ('sa_pv_capital_cost_1',sa_pv_capital_cost_1,'','','Stand-alone PV capital cost (USD/kW) for household systems under 20 W'),
         ('sa_pv_capital_cost_2',sa_pv_capital_cost_2,'','','Stand-alone PV capital cost (USD/kW) for household systems between 21-50 W'),
         ('sa_pv_capital_cost_3',sa_pv_capital_cost_3,'','','Stand-alone PV capital cost (USD/kW) for household systems between 51-100 W'),
         ('sa_pv_capital_cost_4',sa_pv_capital_cost_4,'','','Stand-alone PV capital cost (USD/kW) for household systems between 101-200 W'),
         ('sa_pv_capital_cost_5',sa_pv_capital_cost_5,'','','Stand-alone PV capital cost (USD/kW) for household systems over 200 W'),
         ('mv_line_cost',mv_line_cost,'','','Cost of MV lines in USD/km'),
         ('lv_line_cost',lv_line_cost,'','','Cost of LV lines in USD/km'),
         ('mv_line_capacity',mv_line_capacity,'','','Capacity of MV lines in kW/line'),
         ('lv_line_capacity',lv_line_capacity,'','','Capacity of LV lines in kW/line'),
         ('lv_line_max_length',lv_line_max_length,'','','Maximum length of LV lines (km)'),
         ('hv_line_cost',hv_line_cost,'','','Cost of HV lines in USD/km'),
         ('mv_line_max_length',mv_line_max_length,'','','Maximum length of MV lines (km)'),
         ('hv_lv_transformer_cost',hv_lv_transformer_cost,'','','Cost of HV/MV transformer (USD/unit)'),
         ('mv_increase_rate',mv_increase_rate,'','','percentage'),
         ('max_grid_extension_dist',max_mv_line_dist,'','','Maximum distance that the grid may be extended by means of MV lines'),
         ('annual_new_grid_connections_limit', annual_new_grid_connections_limit,'','','This is the maximum amount of new households that can be connected to the grid in one year (thousands)'),
         ('grid_capacity_limit',annual_grid_cap_gen_limit,'','','This is the maximum generation capacity that can be added to the grid in one year (MW)'),
         ]
labels = ['Variable','Value', 'Source', 'Comments', 'Description']
df_variables = pd.DataFrame.from_records(list1, columns=labels)

In [20]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')

output_dir = filedialog.askdirectory()
output_dir_variables = os.path.join(output_dir, '{}_Variables.csv'.format(scenario_name))
output_dir_results = os.path.join(output_dir, '{}_Results.csv'.format(scenario_name))
output_dir_summaries = os.path.join(output_dir, '{}_Summaries.csv'.format(scenario_name))
output_dir_unelectrified_summaries = os.path.join(output_dir, '{}_Unelectrified_Summaries.csv'.format(scenario_name))
output_dir_electrified_summaries = os.path.join(output_dir, '{}_Electrified_Summaries.csv'.format(scenario_name))
output_dir_combined_summaries = os.path.join(output_dir, '{}_Combined_Summaries.csv'.format(scenario_name))

In [21]:
# Returning the result as a csv file
onsseter.df.to_csv(output_dir_results, index=False)

# Returning the input variables as a csv file
df_variables.to_csv(output_dir_variables, index=False)

# Returning the summaries for initially unelectrified settlements
unelectrified_sumtechs.to_csv(output_dir_unelectrified_summaries, index=True)

# Returning the summaries for initially electrified settlements
electrified_sumtechs.to_csv(output_dir_electrified_summaries, index=True)

# Returning the combined summaries for all settlements
all_sumtechs.to_csv(output_dir_combined_summaries, index=True)